In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("/content/drive/MyDrive/MSDS/Capstone/MSDS Capstone/Spreadsheets from Tim and Data Cleaning/final_df.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/MSDS/Capstone/MSDS Capstone/Spreadsheets from Tim and Data Cleaning/final_df.csv'

In [ ]:
df.head()

In [ ]:
df['abstract'].isnull().sum()

In [ ]:
df = df.dropna()
df['abstract'].isnull().sum()

In [ ]:
df = df.iloc[:, :2]

In [ ]:
df.head()

In [ ]:
# Compute value counts
value_counts = df['fus_related'].value_counts()

# Plot bar graph
plt.bar(value_counts.index, value_counts.values, color='skyblue')

# Add labels and title
plt.xlabel('Values')
plt.ylabel('Counts')
plt.title('Counts of FUS-related articles')
plt.show()

In [ ]:
X = df['abstract']
y = df['fus_related']

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.33,random_state=42)

tfidf = TfidfVectorizer(stop_words='english')


#kinda like scaling here
tfidf.fit(X_train)
X_train_tfidf = tfidf.transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_tfidf = tfidf.fit_transform(X)

In [ ]:
df.isna().sum()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

matrix = cv.fit_transform(df[df['fus_related']==0]['abstract'])
freqs = zip(cv.get_feature_names_out(), matrix.sum(axis=0).tolist()[0])
# sort from largest to smallest
print("Top 20 words used for Non-FUS Articles.")
print(sorted(freqs, key=lambda x: -x[1])[:40])

In [ ]:
cv = CountVectorizer(stop_words='english')

matrix = cv.fit_transform(df[df['fus_related']==1]['abstract'])
freqs = zip(cv.get_feature_names_out(), matrix.sum(axis=0).tolist()[0])
# sort from largest to smallest
print("Top 20 words used for FUS Articles")
print(sorted(freqs, key=lambda x: -x[1])[:40])

In [ ]:
#NAIVE BAYES
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train_tfidf,y_train)

In [ ]:
#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=10000000)

log_model.fit(X_train_tfidf,y_train)

In [ ]:
#SUPPORT VECTOR MACHINE
from sklearn.svm import SVC,LinearSVC

rbf_svc = SVC(probability=True)
rbf_svc.fit(X_train_tfidf,y_train)

In [ ]:
linear_svc = LinearSVC(dual=True)
linear_svc.fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def test_model(model):

    y_preds = model.predict(X_test_tfidf)
    print(confusion_matrix(y_test,y_preds))
    print('\n')
    print(classification_report(y_test,y_preds))

In [ ]:
#NAIVE BAYES
test_model(nb)

In [ ]:
#LOGISTIC REGRESSION
test_model(log_model)

In [ ]:
#SUPPORT VECTOR MACHINE
test_model(rbf_svc)

In [ ]:
#LINEAR SUPPORT VECTOR MACHINE
test_model(linear_svc)

In [ ]:
from sklearn.pipeline import Pipeline #use the pipeline for final model
pipe = Pipeline([('tfidf',TfidfVectorizer()),('svc',SVC(kernel='linear',probability=True))])

pipe.fit(X,y)

In [ ]:
predicted_probs = pipe.predict_proba(['''Transrectal ultrasound is commonly used for guiding prostate cancer biopsy, where 3D ultrasound volume reconstruction is often desired. Current methods for 3D reconstruction from freehand ultrasound scans require external tracking devices to provide spatial information of an ultrasound transducer. This paper presents a novel deep learning approach for sensorless ultrasound volume reconstruction, which efficiently exploits content correspondence between ultrasound frames to reconstruct 3D volumes without external tracking. The underlying deep learning model, deep contextual-contrastive network (DC 2-Net), utilizes self-attention to focus on the speckle-rich areas to estimate spatial movement and then minimizes a margin ranking loss for contrastive feature learning. A case-wise correlation loss over the entire input video helps further smooth the estimated trajectory. We train and validate DC 2-Net on two independent datasets, one containing 619 transrectal scans and the other having 100 transperineal scans. Our proposed approach attained superior performance compared with other methods, with a drift rate of 9.64 % and a prostate Dice of 0.89. The promising results demonstrate the capability of deep neural networks for universal ultrasound volume reconstruction from freehand 2D ultrasound scans without tracking information.'''])

In [ ]:
classes = pipe.classes_

predicted_probabilities = predicted_probs[0]

# Zip the classes with the predicted probabilities
class_probabilities = zip(classes, predicted_probabilities)

# Print the class labels and their corresponding probabilities
for class_label, probability in class_probabilities:
    print(f"Class: {class_label}, Probability: {probability}")

In [ ]:
from joblib import dump,load

dump(pipe,'fus_model.joblib') #saving pipe to model joblib file

In [ ]:
linear = SVC(kernel='linear',probability=True)

from sklearn.model_selection import GridSearchCV

C = np.logspace(0.01,0.1,1,10,100)

param_grid = {'C':C}

grid_model = GridSearchCV(linear,param_grid=param_grid)

grid_model.fit(X_train_tfidf,y_train)

grid_model.best_params_

In [ ]:
def pip(summary):
    prediction = pipe.predict_proba([summary])
    percentage = (prediction[0][0])*100
    return 'Focused Ultrasound Related: ' + str((round(percentage,1)))+'%'

In [ ]:
pip('''The sun dipped low on the horizon, casting long shadows across the golden fields. A gentle breeze rustled through the trees, carrying with it the sweet scent of wildflowers. Birds chirped merrily in the distance, their songs echoing through the tranquil countryside. As evening approached, the sky was painted with hues of orange and pink, a stunning display of nature's beauty. It was a peaceful scene, a moment of quiet reflection in a busy world.''')

'Focused Ultrasound Related: 92.8%'